# Prompting Techniques
Mainly *3 Types* (Kinda):

- Zero-Shot
- Few-Shot & In-Context
- Chain of Thought (CoT)

### Setup & Few Functions

In [1]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
llm = ChatOllama(model="llama3.2")

def createChain(prompt_template):
    """
    Create a LangChain chain with the given prompt template.
    
    Args:
        prompt_template (str): The prompt template string.
    
    Returns:
        LLMChain: A LangChain chain object.
    """
    
    prompt = PromptTemplate.from_template(prompt_template)
    return prompt | llm

## Zero-Shot
Gives _no examples_ to the LLM.

*We'll understand it using 3 Different Methods:*
- Direct Task Specification
- Format Specification
- Multi-step Reasoning

And then we'll be doing a comparative analysis of the *Prompts*

### Direct Task Specification

In [3]:
directTaskPrompt = """Classify the sentiment of the following text as positive, negative, or neutral.
Do not explain your reasoning, just provide the classification.

Text: {text}

Sentiment:"""

directTaskChain = createChain(directTaskPrompt)

# Test the direct task specification
texts = [
    "I absolutely loved the movie! The acting was superb.",
    "The weather today is quite typical for this time of year.",
    "I'm disappointed with the service I received at the restaurant."
]

for text in texts:
    result = directTaskChain.invoke({"text": text}).content
    print(f"Text: {text}")
    print(f"Sentiment: {result}")

Text: I absolutely loved the movie! The acting was superb.
Sentiment: Positive.
Text: The weather today is quite typical for this time of year.
Sentiment: Neutral.
Text: I'm disappointed with the service I received at the restaurant.
Sentiment: Negative


### Format Specification

In [4]:
formatSpecPrompt = """Generate a short news article about {topic}. 
Structure your response in the following format:

Headline: [A catchy headline for the article]

Lead: [A brief introductory paragraph summarizing the key points]

Body: [2-3 short paragraphs providing more details]

Conclusion: [A concluding sentence or call to action]"""

formatSpecChain = createChain(formatSpecPrompt)

# Test the format specification prompting
topic = "The discovery of a new earth-like exoplanet"
result = formatSpecChain.invoke({"topic": topic}).content
print(result)

**Headline:** "New World on the Horizon: Scientists Discover Earth-Like Exoplanet"

**Lead:** In a groundbreaking discovery, a team of astronomers has identified a new exoplanet that bears striking similarities to our own planet Earth. The newly discovered world, dubbed Kepler-186f, is located approximately 490 light-years from us in the constellation Cygnus.

**Body:** According to researchers, Kepler-186f orbits a G-type star - the same type of star that sustains life on Earth - and is situated within its habitable zone. This means that the planet's distance from its star allows it to maintain temperatures suitable for liquid water, a crucial ingredient for life as we know it.

Further analysis suggests that Kepler-186f has a size comparable to that of our own Earth, with some scientists speculating that it may even possess an atmosphere similar to ours. While more research is needed to confirm the presence of life on this distant world, the discovery marks an exciting milestone in t

### Multistep Reasoning

Breaking down into simpler zero-shot steps.

In [5]:
multiStepPrompt = """Analyze the following text for its main argument, supporting evidence, and potential counterarguments. 
Provide your analysis in the following steps:

1. Main Argument: Identify and state the primary claim or thesis.
2. Supporting Evidence: List the key points or evidence used to support the main argument.
3. Potential Counterarguments: Suggest possible objections or alternative viewpoints to the main argument.

Text: {text}

Analysis:"""

multiStepChain = createChain(multiStepPrompt)

# Test the multi-step reasoning approach
text = """While electric vehicles are often touted as a solution to climate change, their environmental impact is not as straightforward as it seems. 
The production of batteries for electric cars requires significant mining operations, which can lead to habitat destruction and water pollution. 
Moreover, if the electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced. 
However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could indeed play a crucial role in combating climate change."""

result = multiStepChain.invoke({"text": text}).content
print(result)

Here's the analysis of the text:

**1. Main Argument:** The primary claim of the text is that electric vehicles are not a straightforward solution to combat climate change, due to their environmental impact during production and charging.

**2. Supporting Evidence:**

* The production of batteries for electric cars requires significant mining operations, leading to habitat destruction and water pollution.
* If electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced.
* However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could play a crucial role in combating climate change.

**3. Potential Counterarguments:**

* Alternative viewpoint 1: While the production of batteries can have negative environmental impacts, these can be mitigated through responsible mining practices and recycling programs.
* Alternative viewpoint 2: Some studies suggest that widesp

### Comparative Analysis

#### Note:
Both Basic & Structured are useful but its important to understand what to use when. 
My Personal Belief is that its better to use *structured prompt*, and structure it for simplicity for a controlled result rather than go for a basic prompt and struggle with unpredictable results every time.

In [6]:
def comparePrompts(task, promptTemplates):
    """
    Compare different prompt templates for the same task.
    
    Args:
        task (str): The task description or input.
        promptTemplates (dict): A dictionary of prompt templates with their names as keys.
    """
    print(f"Task: {task}\n")
    for name, template in promptTemplates.items():
        chain = createChain(template)
        result = chain.invoke({"task": task}).content
        print(f"{name} Prompt Result:")
        print(result)
        print("\n" + "-"*50 + "\n")

In [7]:
task = "Explain conciesly the concept of blockchain technology"

promptTemplates = {
    "Basic": "Explain {task}.",
    "Structured": """Explain {task} by addressing the following points:
1. Definition
2. Key features
3. Real-world applications
4. Potential impact on industries"""
}

comparePrompts(task, promptTemplates)

Task: Explain conciesly the concept of blockchain technology

Basic Prompt Result:
Blockchain technology is a decentralized, digital ledger that records transactions across a network of computers in a secure and transparent way. Here's a concise explanation:

**Key components:**

1. **Decentralized network**: A network of computers (nodes) working together to validate and record transactions.
2. **Digital ledger**: A chain of blocks, each containing a set of transactions, linked together through cryptographic hashes.
3. **Cryptography**: Secure encryption methods protect the integrity and authenticity of transactions.

**How it works:**

1. A new block is added to the ledger with a unique code (hash).
2. Nodes on the network verify the transaction and update their copy of the ledger.
3. Once verified, the block is added to the ledger, creating a permanent record.
4. Each node updates its copy of the ledger, ensuring everyone has the same version.

**Benefits:**

1. **Security**: Transa

## Few-Shot Learning and In-Context Learning

To perform tasks by giving a handful of examples.

### Simple Few-Shot

In [8]:
def fewShotSentimentClassification(inputText):
    fewShotPrompt = PromptTemplate(
        input_variables=["inputText"],
        template="""
        Classify the sentiment as Positive, Negative, or Neutral, only.
        
        Examples:
        Text: I love this product! It's amazing.
        Sentiment: Positive
        
        Text: This movie was terrible. I hated it.
        Sentiment: Negative
        
        Text: The weather today is okay.
        Sentiment: Neutral
        
        Now, classify the following:
        Text: {inputText}
        Sentiment:
        """
    )
    
    chain = fewShotPrompt | llm
    result = chain.invoke(inputText).content

    # Clean up the result
    result = result.strip()
    # Extract only the sentiment label
    if ':' in result:
        result = result.split(':')[1].strip() # We are taking after the colon
    
    return result  # This will now return just "Positive", "Negative", or "Neutral"

testText = "I can't believe how great this new restaurant is!"
result = fewShotSentimentClassification(testText)
print(f"Input: {testText}")
print(f"Predicted Sentiment: {result}")

Input: I can't believe how great this new restaurant is!
Predicted Sentiment: Positive


### Advanced Few-Shot Techniques

#### Multi-Task Learning
Training on multiple related task at the same time. 

In [9]:
def multiTaskFewShot(inputText, task):
    fewShotPrompt = PromptTemplate(
        input_variables=["inputText", "task"],
        template="""
        Perform the specified task on the given text.
        
        Examples:
        Text: I love this product! It's amazing.
        Task: sentiment
        Sentiment: Positive
        
        Text: Bonjour, comment allez-vous?
        Task: language
        Language: French
        
        Now, perform the following task:
        Text: {inputText}
        Task: {task}
        Result:
        """
    )
    
    chain = fewShotPrompt | llm
    return chain.invoke({"inputText": inputText, "task": task}).content

print(multiTaskFewShot("I can't believe how great this is!", "sentiment"))
print(multiTaskFewShot("Guten Tag, wie geht es Ihnen?", "language"))

Sentiment: Positive
The text "Guten Tag, wie geht es Ihnen?" is written in German.


#### In-Context Learning
 allows models to adapt to new tasks based on examples provided in the prompt.


In [10]:
def inContextLearning(taskDescription, examples, inputText):
    exampleText = "".join([f"Input: {e['input']}\nOutput: {e['output']}\n\n" for e in examples])
    
    in_context_prompt = PromptTemplate(
        input_variables=["taskDescription", "examples", "inputText"],
        template="""
        Task: {taskDescription}
        
        Examples:
        {examples}
        
        Now, perform the task on the following input:
        Input: {inputText}
        Output:
        """
    )
    
    chain = in_context_prompt | llm
    return chain.invoke({"taskDescription": taskDescription, "examples": exampleText, "inputText": inputText}).content

taskDesc = "Convert the given text to pig latin. Give Output Only."
examples = [
    {"input": "hello", "output": "ellohay"},
    {"input": "apple", "output": "appleay"}
]
testInput = "python"

result = inContextLearning(taskDesc, examples, testInput)
print(f"Input: {testInput}")
print(f"Output: {result}")

Input: python
Output: ythonpay


### Evaluation

In [11]:
def evaluateModel(modelFunc, testCases):
    '''
    Evaluate the model on a set of test cases.

    Args:
    modelFunc: The function that makes predictions.
    testCases: A list of dictionaries, where each dictionary contains an "input" text and a "label" for the input.

    Returns:
    The accuracy of the model on the test cases. 
    '''
    correct = 0
    total = len(testCases)
    
    for case in testCases:
        inputText = case['input']
        trueLabel = case['label']
        prediction = modelFunc(inputText).strip()
        
        isCorrect = prediction.lower() == trueLabel.lower()
        correct += int(isCorrect)
        
        print(f"Input: {inputText}")
        print(f"Predicted: {prediction}")
        print(f"Actual: {trueLabel}")
        print(f"Correct: {isCorrect}\n")
    
    accuracy = correct / total
    return accuracy

testCases = [
    {"input": "This product exceeded my expectations!", "label": "Positive"},
    {"input": "I'm utterly disappointed with the service.", "label": "Negative"},
    {"input": "The temperature today is 72 degrees.", "label": "Neutral"}
]

accuracy = evaluateModel(fewShotSentimentClassification, testCases)
print(f"Model Accuracy: {accuracy:.2f}")

Input: This product exceeded my expectations!
Predicted: Positive
Actual: Positive
Correct: True

Input: I'm utterly disappointed with the service.
Predicted: Negative
Actual: Negative
Correct: True

Input: The temperature today is 72 degrees.
Predicted: Neutral
Actual: Neutral
Correct: True

Model Accuracy: 1.00


#### Note:

Accuracy changes with each re-run. Raising Questions of reliability.

## Chain of Thoughts

 encourages AI models to break down complex problems into step-by-step reasoning processes.

### Basic Chain of Thought: Comparision

In [12]:
# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely: {question}."
)

# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question step by step concisely: {question}"
)

# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Example question
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"

# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
To find the average speed, divide the distance traveled by the time taken:

Average Speed = Distance / Time
= 120 km / 2 hours
= 60 km/h.

Chain of Thought Response:
Here's the solution:

1. Distance traveled = 120 km
2. Time taken = 2 hours
3. Average speed = Total distance / Time taken = 120 km / 2 hours = 60 km/h


### Advanced Chain of Thought

Encouraging the LLM by giving the steps to follow in the prompt itself.

In [13]:
advanced_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Solve the following problem step by step. For each step:
1. State what you're going to calculate
2. Write the formula you'll use (if applicable)
3. Perform the calculation
4. Explain the result

Question: {question}

Solution:"""
)

advanced_cot_chain = advanced_cot_prompt | llm

complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

advanced_cot_response = advanced_cot_chain.invoke(complex_question).content
print(advanced_cot_response)

Here's the step-by-step solution:

**Step 1: State what we're going to calculate**
We need to find the average speed for the entire journey.

**Step 2: Write the formula you'll use (if applicable)**
The formula for average speed is:
Average Speed = Total Distance / Total Time

Since time can be calculated using the formula:
Time = Distance / Speed
we will first calculate the total distance and individual times, then add them together to find the total time.

**Step 3: Perform the calculations**

**For the first part of the journey (150 km at 60 km/h):**
Distance = 150 km
Speed = 60 km/h

Time = Distance / Speed
= 150 km / 60 km/h
= 2.5 hours

**For the second part of the journey (100 km at 50 km/h):**
Distance = 100 km
Speed = 50 km/h

Time = Distance / Speed
= 100 km / 50 km/h
= 2 hours

**Total distance and time:**
Total Distance = 150 km + 100 km = 250 km
Total Time = 2.5 hours + 2 hours = 4.5 hours

**Step 4: Calculate the average speed using the formula**
Average Speed = Total Dis

### Comparitive Analysis: Basic VS COT

Chain of Thoughts method shines on complicated multi-step challenging Qs.

In [16]:
challenging_question = """
A cylindrical water tank with a radius of 1.5 meters and a height of 4 meters is 2/3 full. 
If water is being added at a rate of 10 liters per minute, how long will it take for the tank to overflow? 
Give your answer in hours and minutes, rounded to the nearest minute. 
(Use 3.14159 for π and 1000 liters = 1 cubic meter)"""

standard_response = standard_chain.invoke(challenging_question).content
cot_response = advanced_cot_chain.invoke(challenging_question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
To solve this problem, we need to calculate the volume of water in the tank when it is full, then find the remaining volume that needs to be added.

The formula for the volume of a cylinder is:

V = πr^2h

where V is the volume, π is pi (approximately 3.14159), r is the radius, and h is the height.

First, calculate the total capacity of the tank when it's full:
V = π(1.5)^2(4)
= 3.14159 * 2.25 * 4
≈ 28.27 cubic meters

Since 1 cubic meter is equal to 1000 liters, the full capacity is:
28.27 cubic meters * 1000 liters/cubic meter ≈ 28270 liters

The tank is currently 2/3 full, so we need to calculate the current volume of water in the tank:
Current volume = (2/3) * 28270 liters
≈ 18846.67 liters

Now, let's find out how much more water needs to be added for the tank to overflow:
Remaining volume = Total capacity - Current volume
= 28270 liters - 18846.67 liters
≈ 9423.33 liters

Since water is being added at a rate of 10 liters per minute, we can calculate the time i

*Note:* Through multiple re-runs, the CoT prompt consistently provides correct answers, whereas the standard prompt only achieves this intermittently.

### Logical Reasoning: COT Application

In [15]:
logical_reasoning_prompt = PromptTemplate(
    input_variables=["scenario"],
    template="""Analyze the following logical puzzle thoroughly. Follow these steps in your analysis:

List the Facts:

Summarize all the given information and statements clearly.
Identify all the characters or elements involved.
Identify Possible Roles or Conditions:

Determine all possible roles, behaviors, or states applicable to the characters or elements (e.g., truth-teller, liar, alternator).
Note the Constraints:

Outline any rules, constraints, or relationships specified in the puzzle.
Generate Possible Scenarios:

Systematically consider all possible combinations of roles or conditions for the characters or elements.
Ensure that all permutations are accounted for.
Test Each Scenario:

For each possible scenario:
Assume the roles or conditions you've assigned.
Analyze each statement based on these assumptions.
Check for consistency or contradictions within the scenario.
Eliminate Inconsistent Scenarios:

Discard any scenarios that lead to contradictions or violate the constraints.
Keep track of the reasoning for eliminating each scenario.
Conclude the Solution:

Identify the scenario(s) that remain consistent after testing.
Summarize the findings.
Provide a Clear Answer:

State definitively the role or condition of each character or element.
Explain why this is the only possible solution based on your analysis.
Scenario:

{scenario}

Analysis:""")

logical_reasoning_chain = logical_reasoning_prompt | llm

logical_puzzle = """In a room, there are three people: Amy, Bob, and Charlie. 
One of them always tells the truth, one always lies, and one alternates between truth and lies. 
Amy says, 'Bob is a liar.' 
Bob says, 'Charlie alternates between truth and lies.' 
Charlie says, 'Amy and I are both liars.' 
Determine the nature (truth-teller, liar, or alternator) of each person."""

logical_reasoning_response = logical_reasoning_chain.invoke(logical_puzzle).content
print(logical_reasoning_response)

**List of Facts:**

The given information is as follows:

1. There are three people in a room: Amy, Bob, and Charlie.
2. One person always tells the truth, one person always lies, and one person alternates between truth and lies.
3. Amy says, 'Bob is a liar.'
4. Bob says, 'Charlie alternates between truth and lies.'
5. Charlie says, 'Amy and I are both liars.'

**Characters or Elements:**

The three characters involved are:

1. Amy
2. Bob
3. Charlie

**Possible Roles or Conditions:**

The possible roles or conditions for the characters are:

1. Truth-teller (always tells the truth)
2. Liar (always lies)
3. Alternator (alternates between truth and lies)

**Constraints:**

The constraints specified in the puzzle are:

1. One person always tells the truth.
2. One person always lies.
3. One person alternates between truth and lies.
4. The statements made by each person must be consistent with their role or condition.

**Generate Possible Scenarios:**

Let's consider all possible combinatio